In [15]:
import pandas as pd
import seaborn as sns
import re
import numpy as np

In [16]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [17]:
def get_title(name):
    match = re.search(' ([A-Za-z]+)\.', name)
    if match:
        return match.group()
    else:
        return NaN

In [18]:
def guess_age(cols):
    age = cols[0]
    title = cols[1]
    if pd.isnull(age):
        if title == 0:
            return 32.368090
        elif title == 0.1:
            return 21.773973
        elif title == 0.2:
            return 35.898148
        elif title == 0.3:
            return 42.384615
        elif title == 10.0:
            return 4.574167
    else:
        return age
#함수로 하는게 비효율적이고 코드가 좋지 못한듯.

In [19]:
sex_map = {"male":0,"female":1}
embarked_map = {'S':0,'Q':1,'C':2}
boolean_map = {False:0,True:1}
title_map = {' Mr.':0, ' Miss.':0.1, ' Mrs.':0.2, ' Master.':10}#해당 수치 조절 필요할듯.. Title이 가지는 의미에 대해 같이 고민해봅시당 
train['Sex'] = train['Sex'].map(sex_map)

train['Embarked'] = train['Embarked'].map(embarked_map)
train['Embarked'] = train['Embarked'].fillna(0)#누락된값이 2개밖에 없어서 그냥 S로 했습니다.

train['Cabin'] = train['Cabin'].notnull()
train['Cabin'] = train['Cabin'].map(boolean_map)

train['Title'] = train['Name'].map(get_title)
train['Title'] = train['Title'].map(title_map)
train['Title'] = train['Title'].fillna(0.3)

train['Age'] = train[['Age','Title']].apply(guess_age,axis=1)#비어있는 Age에 대해 각 title별 평균나이 적용 했어요

#train[['Title','Age']].groupby(['Title']).mean() title별 Age의 평균
test['Sex'] = test['Sex'].map(sex_map)

test['Embarked'] = test['Embarked'].map(embarked_map)
test['Embarked'] = test['Embarked'].fillna(0)

test['Cabin'] = test['Cabin'].notnull()
test['Cabin'] = test['Cabin'].map(boolean_map)#Cabin의 경우에 기록 유무에만 따라 0 1 로 했어요

test['Title'] = test['Name'].map(get_title)
test['Title'] = test['Title'].map(title_map)
test['Title'] = test['Title'].fillna(0.3)#이것두 Title의 조정 수치

test['Age'] = test[['Age','Title']].apply(guess_age,axis=1)

train.drop('PassengerId',axis=1,inplace=True)
train.drop('Name',axis=1,inplace=True)
train.drop('Ticket',axis=1,inplace=True)

test.drop('Name',axis=1,inplace=True)
test.drop('Ticket',axis=1,inplace=True)

test_data=test.drop('PassengerId',axis=1).copy()

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       891 non-null int64
Embarked    891 non-null float64
Title       891 non-null float64
dtypes: float64(4), int64(6)
memory usage: 69.7 KB


In [21]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from tensorflow.contrib import learn

In [22]:
y_data = train['Survived']
x_data = train[['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked','Title']]

#x_train = x_data
#y_train = y_data
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2,random_state=42)#교차검증 데이터셋

In [ ]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(x_train)
classifier = learn.DNNClassifier(feature_columns=feature_columns, hidden_units=[10, 20, 10], n_classes=2)
classifier.fit(x_train, y_train,steps=3000) #hidden layer 3개의 초기 모델

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000210ACC3EDA0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ljh_9\\AppData\\Local\\Temp\\tmpd9g5mpv7'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving check

In [12]:
predicted = classifier.predict(x_test)
print(accuracy_score(y_test,list(predicted)))
#교차검증 확인

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ljh_9\AppData\Local\Temp\tmpk669dfhm\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0.8268156424581006


In [13]:
result = classifier.predict(test_data)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ljh_9\AppData\Local\Temp\tmpk669dfhm\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [14]:
submission = pd.DataFrame({
    #"PassengerId" : test.index+892,
    "PassengerId" : test["PassengerId"],
    "Survived" : list(result)
})
submission.to_csv('submission.csv',index=False)
#초기 모델 교차검증 0.8212290502793296 , Kaggle Score 0.78468(10000명 중 3944등)